In [158]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [159]:
past_data = pickle.load(open('past_data.pkl', 'rb'))


In [160]:
xgb_model = pickle.load(open('xgb_model_final.pkl', 'rb'))

In [161]:
def predict_pickups(input_df,past_data):
    # Converting Time stamps to UNIX time stamp format
    # "YYYY-MM-DD HH:MM:SS" into unix time stamp

    def convert_to_unix(s):
        s = str(s)
        return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())

    #Refer:https://www.unixtimestamp.com/
    # 1640975400 : 2022-01-01 00:00:00 
    # 1643653800 : 2022-02-01 00:00:00 
    # 1646073000 : 2022-03-01 00:00:00

    # 1672511400 : 2023-01-01 00:00:00 
    # 1675189800 : 2023-02-01 00:00:00 
    # 1677609000 : 2023-03-01 00:00:00

    def add_pickup_bins(timestamp):
        """
        Adds pickup bins to the given DataFrame

        Args:
            frame (DataFrame): The DataFrame to which pickup bins will be added.

        Returns:
            DataFrame: The updated DataFrame with pickup bins added.
        """
        
        start_pickup_unix = 1672511400
        tenminutewise_binned_unix_pickup_times = (int((timestamp - start_pickup_unix) / 600))
        bin_number = tenminutewise_binned_unix_pickup_times

        return bin_number
    
    # converting to unix time stamp
    input_df['PU_timestamp'] = input_df['PU_timestamp'].apply(convert_to_unix)
    # adding pickup bin number 
    input_df['bin_number'] = input_df['PU_timestamp'].apply(add_pickup_bins) 
    # Calculating the row number for the given bin number
    input_df['row_number'] = input_df['bin_number']  + (input_df['locationID'] * 12960)
    # Calculating the row number for the given bin number
    input_df['row_number'] = input_df['row_number'] - 5*(input_df['locationID']+1)
    
    
    # getting the feature row for the given bin number
    feature_row = past_data.iloc[input_df['row_number']] 
    # reseting the index
    feature_row = feature_row.reset_index(drop=True)
    # print(feature_row)
    
    # predicting the number of pickups
    y_pred = xgb_model.predict(feature_row) 
    output = round(y_pred[0])
    
    
    return output
    

In [162]:
# API input
input_locationID = 264
input_timestamp = '2023-02-01 23:59:00'

data = {'locationID': [input_locationID],
        'PU_timestamp': [input_timestamp]}

input_df = pd.DataFrame(data)

In [163]:
print("Predicted number of Pickups:",predict_pickups(input_df,past_data))

Predicted number of Pickups: 1
